In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from tqdm.auto import tqdm
from skimage.metrics import peak_signal_noise_ratio
from IPython import display
from sr_model import CNNSuperResolutionModel, VGGLoss
import cv2

In [2]:
model = CNNSuperResolutionModel().cuda()
model.load_state_dict(torch.load('model.pt'))

<All keys matched successfully>

In [6]:
img = cv2.imread("urban_dataset_15cm/1.png")

print(img.shape)

img_copy = img.copy()

y_chan = cv2.cvtColor(img, cv2.COLOR_BGR2YCR_CB)[:, :, 0] #Y channel

blur_kernel = np.genfromtxt('PSF_E10x2.csv', delimiter=';')

blur_y_chann = cv2.filter2D(y_chan, -1, blur_kernel)

width, height = y_chan.shape

chunk_size = 256

num_chunks_x = height // chunk_size
num_chunks_y = width // chunk_size


for i in tqdm(range(num_chunks_x)):
    for j in range(num_chunks_y):

        left = i * chunk_size
        top = j * chunk_size
        right = left + chunk_size
        bottom = top + chunk_size
        
        hr_patch = y_chan[top:bottom, left:right]
        blur_patch = cv2.filter2D(hr_patch, -1, blur_kernel)
        
        y_hat = model(transforms.ToTensor()(blur_patch).cuda().unsqueeze(0)).squeeze(0)
        
        img_copy[top:bottom, left:right, 0] = y_hat.cpu().detach().numpy()

cv2.imwrite("urban_dataset_15cm/1_sr.png", img_copy)
        
        
        
        

(1419, 1672, 3)


  0%|          | 0/6 [00:00<?, ?it/s]

True